In [1]:
import requests
import pandas as pd
import json 
import numpy as np

In [2]:
#Definition call all the APIs needed 

def retrieve(GET):
    #URL to get API with name to get specific table
    url = f"https://pokemon-go1.p.rapidapi.com/{GET}.json"
    #Dictionary that holds the API host and Key
    headers = {
    'x-rapidapi-host': "pokemon-go1.p.rapidapi.com",
    'x-rapidapi-key': "76d2396840mshc562c26618d33a2p1fb1e6jsn95b66aa3ea3c"
    }
    #putting together the URL and Dictionary(API keys) to pull from API
    response = requests.request("GET", url, headers=headers).json()
    #create a pandas DF out of the API Call
    df = pd.json_normalize(response)
#     return DataFrame
    return df

In [3]:
# Choosen APIs I want to have called. 
GETS = ['pokemon_stats','pokemon_max_cp','fast_moves','charged_moves','pokemon_encounter_data','pokemon_types']

# Create a dictionary that will hold the the API Calls
RapidAPI_dic = {}

# Run through the definition with all of the APIs that I chose and store them in the dictionary
for i in GETS:
    RapidAPI_dic[i] = retrieve(i)


In [4]:
#as seen when this Data Frame is run there are multiple forms of the same pokemon.
Rapid_Stats = RapidAPI_dic['pokemon_stats']
Rapid_Stats

,base_attack,base_defense,base_stamina,form,pokemon_id,pokemon_name
0,118,111,128,Fall_2019,1,Bulbasaur
1,118,111,128,Normal,1,Bulbasaur
2,118,111,128,Purified,1,Bulbasaur
3,118,111,128,Shadow,1,Bulbasaur
4,151,143,155,Normal,2,Ivysaur
...,...,...,...,...,...,...
1117,195,162,172,Purified,863,Perrserker
1118,195,162,172,Shadow,863,Perrserker
1119,248,176,158,Normal,865,Sirfetch’d
1120,248,176,158,Purified,865,Sirfetch’d


In [5]:
# First we must see if any of the pokemon stats vary over the different forms.
# create a total stats column to make comparison against each form easier
Rapid_Stats['total_stats'] = Rapid_Stats['base_defense'] + Rapid_Stats['base_attack'] 

#Take all pokemon with Normal Forms and put them in a DataFrame
Normal = Rapid_Stats['form'] == 'Normal'
Normal = Rapid_Stats[Normal]
Normal
# Create variable to hold total stat number
stats = 0
# 
counter = 1
#create a for loop to check each pokemon form against itself
for index, row in Rapid_Stats.iterrows():
    if row[5] in Normal.values:
        stats = Normal.loc[Normal["pokemon_name"]==row[5]]['total_stats'].values[0]
        if stats != row[6]:
            
            newindex = Normal[-1:].index[0]+ 1
            Normal.loc[index, 'base_attack'] = row[0]
            Normal.loc[index,'base_defense'] = row[1]
            Normal.loc[index,'base_stamina'] = row[2]
            Normal.loc[index,'form'] = row[3]
            Normal.loc[index,'pokemon_id'] = row[4]
            Normal.loc[index,'pokemon_name'] = row[5]
            Normal.loc[index,'total_stats'] = row[6]
#             print(newindex)
            counter += 1
    elif row[5] not in Normal.values:
        newindex = Normal[-1:].index[0]+ 1
        Normal.loc[index, 'base_attack'] = row[0]
        Normal.loc[index,'base_defense'] = row[1]
        Normal.loc[index,'base_stamina'] = row[2]
        Normal.loc[index,'form'] = row[3]
        Normal.loc[index,'pokemon_id'] = row[4]
        Normal.loc[index,'pokemon_name'] = row[5]
        Normal.loc[index,'total_stats'] = row[6]
#         print(f'{newindex} Not Normal')
        counter += 1
        
print(counter)

C:\Users\Lenny Ovo\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


51


In [6]:
# Normal.loc[Normal["pokemon_name"]== 'Kyurem']
Normal = Normal.sort_index()
Normal

,base_attack,base_defense,base_stamina,form,pokemon_id,pokemon_name,total_stats
1,118.0,111.0,128.0,Normal,1.0,Bulbasaur,229.0
4,151.0,143.0,155.0,Normal,2.0,Ivysaur,294.0
8,198.0,189.0,190.0,Normal,3.0,Venusaur,387.0
12,116.0,93.0,118.0,Normal,4.0,Charmander,209.0
15,158.0,126.0,151.0,Normal,5.0,Charmeleon,284.0
...,...,...,...,...,...,...,...
1111,118.0,99.0,130.0,Normal,808.0,Meltan,217.0
1112,226.0,190.0,264.0,Normal,809.0,Melmetal,416.0
1113,180.0,194.0,212.0,Galarian,862.0,Obstagoon,374.0
1116,195.0,162.0,172.0,Galarian,863.0,Perrserker,357.0


In [7]:
# Adding Max CP to the Data Frame
Rapid_CP = RapidAPI_dic['pokemon_max_cp']
Rapid_CP = Rapid_CP.drop(['form' , 'pokemon_id' , 'pokemon_name'] , axis = 1)
Rapid = pd.concat([Normal , Rapid_CP], axis = 1 , join = 'inner')


#Pulling in types of each pokemon
Rapid_Types = RapidAPI_dic['pokemon_types']
Rapid_Types
# there is two types inside of the type column, it has to be separated into two seperate columns
types =  pd.DataFrame(Rapid_Types['type'].tolist())

Rapid_Final = pd.concat([ Rapid , types], axis = 1 , join = 'inner')

Rapid_Final = Rapid_Final.rename(columns={ 0: "Primary_Type", 1: "Secondary_Type" , 'base_attack' : 'Attack' , 
                           'base_defense' : 'Defense' , 'base_stamina' : 'Stamina' , 'pokemon_id': 'Pokemon_ID' , 
                           'total_stats' : 'Total_Stats' , 'max_cp' : 'Max_CP' , 'form': 'Form' ,'pokemon_name' : 'Name'})
Rapid_Final = Rapid_Final[['Name' , 'Pokemon_ID' , 'Primary_Type' , 'Secondary_Type', 'Attack' , 'Defense' , 'Stamina' , 'Total_Stats' , 'Max_CP' , 'Form']]

In [8]:
Rapid_Final

,Name,Pokemon_ID,Primary_Type,Secondary_Type,Attack,Defense,Stamina,Total_Stats,Max_CP,Form
1,Bulbasaur,1.0,Grass,Poison,118.0,111.0,128.0,229.0,1115,Normal
4,Ivysaur,2.0,Grass,Poison,151.0,143.0,155.0,294.0,1699,Normal
8,Venusaur,3.0,Grass,Poison,198.0,189.0,190.0,387.0,2720,Normal
12,Charmander,4.0,Fire,None,116.0,93.0,118.0,209.0,980,Normal
15,Charmeleon,5.0,Fire,None,158.0,126.0,151.0,284.0,1653,Normal
...,...,...,...,...,...,...,...,...,...,...
1111,Meltan,808.0,Steel,None,118.0,99.0,130.0,217.0,1068,Normal
1112,Melmetal,809.0,Steel,None,226.0,190.0,264.0,416.0,3599,Normal
1113,Obstagoon,862.0,Dark,Normal,180.0,194.0,212.0,374.0,2652,Galarian
1116,Perrserker,863.0,Steel,None,195.0,162.0,172.0,357.0,2386,Galarian


In [9]:
# Adding Max CP to the Data Frame
Rapid_encounter = RapidAPI_dic['pokemon_encounter_data']
Both = pd.concat([Rapid_Final , Rapid_encounter], axis = 1 , join = 'inner')

Both = Both[["Pokemon_ID" , "Name" , "Primary_Type" , "Secondary_Type" , "Form", "Max_CP" ,  "Stamina" , "Attack" , "Defense" ,
      "Total_Stats" ,   "attack_probability" , "base_capture_rate" , "base_flee_rate" , "dodge_probability" ,
      "max_pokemon_action_frequency" , "min_pokemon_action_frequency" ]]

Both = Both.rename(columns = {'attack_probability' : 'Attack_Probability' , 'base_capture_rate' : 'Base_Capture_Rate' , 
                               'base_flee_rate' : 'Base_Flee_Rate' , 'dodge_probability' : 'Dodge_Probability' , 'max_pokemon_action_frequency': 
                               'Max_Action_Frequency' , 'min_pokemon_action_frequency' : 'Min_Action_Frequency'})

In [10]:
Rapid_Split = np.split(Both, [5,10] , axis =1)
Rapid_Split[1]

,Max_CP,Stamina,Attack,Defense,Total_Stats
1,1115,128,118,111,229
4,1699,155,151,143,294
8,2720,190,198,189,387
12,980,118,116,93,209
15,1653,151,158,126,284
...,...,...,...,...,...
1111,1068,130,118,99,217
1112,3599,264,226,190,416
1113,2652,212,180,194,374
1116,2386,172,195,162,357


In [11]:
Battle_R = Rapid_Split[1]
Battle_R['Primary_Key'] = Battle_R.index
Battle_R = Battle_R.drop_duplicates( subset =  ['Max_CP' , 'Stamina' , 'Attack' , 'Defense' , 'Total_Stats'])
Battle_R

,Max_CP,Stamina,Attack,Defense,Total_Stats,Primary_Key
1,1115,128,118,111,229,1
4,1699,155,151,143,294,4
8,2720,190,198,189,387,8
12,980,118,116,93,209,12
15,1653,151,158,126,284,15
...,...,...,...,...,...,...
1111,1068,130,118,99,217,1111
1112,3599,264,226,190,416,1112
1113,2652,212,180,194,374,1113
1116,2386,172,195,162,357,1116


In [12]:
Both

,Pokemon_ID,Name,Primary_Type,Secondary_Type,Form,Max_CP,Stamina,Attack,Defense,Total_Stats,Attack_Probability,Base_Capture_Rate,Base_Flee_Rate,Dodge_Probability,Max_Action_Frequency,Min_Action_Frequency
1,1.0,Bulbasaur,Grass,Poison,Normal,1115,128.0,118.0,111.0,229.0,0.1,0.20,0.10,0.15,1.6,0.2
4,2.0,Ivysaur,Grass,Poison,Normal,1699,155.0,151.0,143.0,294.0,0.1,0.10,0.07,0.15,1.6,0.2
8,3.0,Venusaur,Grass,Poison,Normal,2720,190.0,198.0,189.0,387.0,0.2,0.05,0.05,0.15,1.6,0.2
12,4.0,Charmander,Fire,None,Normal,980,118.0,116.0,93.0,209.0,0.1,0.20,0.10,0.15,1.6,0.2
15,5.0,Charmeleon,Fire,None,Normal,1653,151.0,158.0,126.0,284.0,0.1,0.10,0.07,0.15,1.6,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,808.0,Meltan,Steel,None,Normal,1068,130.0,118.0,99.0,217.0,0.1,0.30,0.00,0.15,1.6,0.2
1112,809.0,Melmetal,Steel,None,Normal,3599,264.0,226.0,190.0,416.0,0.1,0.30,0.00,0.15,1.6,0.2
1113,862.0,Obstagoon,Dark,Normal,Galarian,2652,212.0,180.0,194.0,374.0,0.3,0.04,0.04,0.15,1.6,0.2
1116,863.0,Perrserker,Steel,None,Galarian,2386,172.0,195.0,162.0,357.0,0.3,0.10,0.07,0.45,1.6,0.2


In [13]:
Both['Battle_FK'] = 0


for index, row in Both.iterrows():
    for index2, row2 in Battle_R.iterrows():
        if row[5] == row2[0] and row[6] == row2[1] and row[7] == row2[2] and row[8] == row2[3] and row[9] == row2[4]:
            Both.loc[index, 'Battle_FK'] = int(row2[5])
           

In [14]:
Both

,Pokemon_ID,Name,Primary_Type,Secondary_Type,Form,Max_CP,Stamina,Attack,Defense,Total_Stats,Attack_Probability,Base_Capture_Rate,Base_Flee_Rate,Dodge_Probability,Max_Action_Frequency,Min_Action_Frequency,Battle_FK
1,1.0,Bulbasaur,Grass,Poison,Normal,1115,128.0,118.0,111.0,229.0,0.1,0.20,0.10,0.15,1.6,0.2,1
4,2.0,Ivysaur,Grass,Poison,Normal,1699,155.0,151.0,143.0,294.0,0.1,0.10,0.07,0.15,1.6,0.2,4
8,3.0,Venusaur,Grass,Poison,Normal,2720,190.0,198.0,189.0,387.0,0.2,0.05,0.05,0.15,1.6,0.2,8
12,4.0,Charmander,Fire,None,Normal,980,118.0,116.0,93.0,209.0,0.1,0.20,0.10,0.15,1.6,0.2,12
15,5.0,Charmeleon,Fire,None,Normal,1653,151.0,158.0,126.0,284.0,0.1,0.10,0.07,0.15,1.6,0.2,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,808.0,Meltan,Steel,None,Normal,1068,130.0,118.0,99.0,217.0,0.1,0.30,0.00,0.15,1.6,0.2,1111
1112,809.0,Melmetal,Steel,None,Normal,3599,264.0,226.0,190.0,416.0,0.1,0.30,0.00,0.15,1.6,0.2,1112
1113,862.0,Obstagoon,Dark,Normal,Galarian,2652,212.0,180.0,194.0,374.0,0.3,0.04,0.04,0.15,1.6,0.2,1113
1116,863.0,Perrserker,Steel,None,Galarian,2386,172.0,195.0,162.0,357.0,0.3,0.10,0.07,0.45,1.6,0.2,1116


In [15]:
Catch_R = Rapid_Split[2]
Catch_R['Primary_Key'] = Catch_R.index
Catch_R = Catch_R.drop_duplicates( subset =  ['Attack_Probability' , 'Base_Capture_Rate' , 'Base_Flee_Rate' , 'Dodge_Probability' ,  
                               'Max_Action_Frequency' , 'Min_Action_Frequency'])
Catch_R

,Attack_Probability,Base_Capture_Rate,Base_Flee_Rate,Dodge_Probability,Max_Action_Frequency,Min_Action_Frequency,Primary_Key
1,0.1,0.2,0.1,0.15,1.6,0.2,1
4,0.1,0.1,0.07,0.15,1.6,0.2,4
8,0.2,0.05,0.05,0.15,1.6,0.2,8
33,0.1,0.5,0.2,0.15,1.6,0.2,33
34,0.1,0.25,0.09,0,1.6,0.2,34
...,...,...,...,...,...,...,...
1081,0.7,0.2,0.07,0.2,1.6,0.2,1081
1086,0.1,0.7,0.15,0.15,1.6,0.2,1086
1111,0.1,0.3,0,0.15,1.6,0.2,1111
1113,0.3,0.04,0.04,0.15,1.6,0.2,1113


In [16]:
Both['Catch_FK'] = 0


for index, row in Both.iterrows():
    for index2, row2 in Catch_R.iterrows():
        if row[10] == row2[0] and row[11] == row2[1] and row[12] == row2[2] and row[13] == row2[3] and row[14] == row2[4] and row[15] == row2[5]:
            Both.loc[index, 'Catch_FK'] = int(row2[6])
    

In [17]:
Rapid_Names = Both

In [18]:
Catch_R

,Attack_Probability,Base_Capture_Rate,Base_Flee_Rate,Dodge_Probability,Max_Action_Frequency,Min_Action_Frequency,Primary_Key
1,0.1,0.2,0.1,0.15,1.6,0.2,1
4,0.1,0.1,0.07,0.15,1.6,0.2,4
8,0.2,0.05,0.05,0.15,1.6,0.2,8
33,0.1,0.5,0.2,0.15,1.6,0.2,33
34,0.1,0.25,0.09,0,1.6,0.2,34
...,...,...,...,...,...,...,...
1081,0.7,0.2,0.07,0.2,1.6,0.2,1081
1086,0.1,0.7,0.15,0.15,1.6,0.2,1086
1111,0.1,0.3,0,0.15,1.6,0.2,1111
1113,0.3,0.04,0.04,0.15,1.6,0.2,1113


In [19]:
Battle_R

,Max_CP,Stamina,Attack,Defense,Total_Stats,Primary_Key
1,1115,128,118,111,229,1
4,1699,155,151,143,294,4
8,2720,190,198,189,387,8
12,980,118,116,93,209,12
15,1653,151,158,126,284,15
...,...,...,...,...,...,...
1111,1068,130,118,99,217,1111
1112,3599,264,226,190,416,1112
1113,2652,212,180,194,374,1113
1116,2386,172,195,162,357,1116
